<a href="https://colab.research.google.com/github/SnSabu/congressional-analysis/blob/main/Flight_Data_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepping input data

In [2]:
import pandas as pd

In [3]:
Office_Airports = pd.read_csv("Closest_airport_to_hubs.csv")
Emp_Airport = pd.read_csv("closest_airports_employees.csv")

,hub,Airport 1,latitude 1,longitude 1,Distance 1,Airport 2,latitude 2,longitude 2,Distance 2
0,Atlanta,ATL,33.636700,-84.428101,97.771765,BNA,36.124500,-86.678200,281.048239
1,Bellevue,SEA,47.449162,-122.311134,20.702747,PDX,45.588699,-122.598000,227.724885
2,Chicago,ORD,41.978600,-87.904800,52.914156,MKE,42.947201,-87.896599,56.923670
3,Dallas,DFW,32.896801,-97.038002,24.744045,AUS,30.197535,-97.662015,298.638768
4,Denver,DEN,39.861698,-104.672996,30.233466,SLC,40.785749,-111.979746,604.938992


In [40]:
hub_airport = pd.concat([Office_Airports['Airport 1'], Office_Airports['Airport 2']])
hub_airport = pd.DataFrame(hub_airport, columns=['Destination Airport'])

emp_airport = pd.concat([pd.Series(Emp_Airport['Airport 1'].unique()), pd.Series(Emp_Airport['Airport 2'].unique())])
emp_airport = pd.DataFrame(emp_airport, columns=['Origin Airport'])


hub_airport['key'] = 1
emp_airport['key'] = 1
airport_combinations = pd.merge(emp_airport, hub_airport, on ='key').drop("key", 1)
airport_combinations.describe

<ipython-input-40-c14fcf9bbd4e>:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  airport_combinations = pd.merge(emp_airport, hub_airport, on ='key').drop("key", 1)


<bound method NDFrame.describe of      Origin Airport Destination Airport
0               BOS                 ATL
1               BOS                 SEA
2               BOS                 ORD
3               BOS                 DFW
4               BOS                 DEN
...             ...                 ...
2575            FAI                 CVG
2576            FAI                 EWR
2577            FAI                 SFO
2578            FAI                 PDX
2579            FAI                 SFO

[2580 rows x 2 columns]>

Using Duffel

In [41]:
!pip install duffel-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 KB 3.8 MB/s eta 0:00:00


In [42]:
from duffel_api import Duffel

#duffel_test_mSO5C-rVg_mIuSj_co7IND-CIcuZ_ak0I8M1X-jas3A
access_token = 'duffel_test_mSO5C-rVg_mIuSj_co7IND-CIcuZ_ak0I8M1X-jas3A'
client = Duffel(access_token = access_token)

In [44]:
from duffel_api import Duffel

client = Duffel(access_token = 'duffel_test_mSO5C-rVg_mIuSj_co7IND-CIcuZ_ak0I8M1X-jas3A')

offer_requests = client.offer_requests.list()
for offer_request in offer_requests:
    print(offer_request.id)

orq_0000AT23PKyBtmqYbqqjBY
orq_0000AT1GdRODdT0LbBSTbg
orq_0000AT1FlCoHvClBxuS3xg
orq_0000AT1Fj50sLbc1wvZqym
orq_0000AT1ATlsghLzoUvd1Ye
orq_0000AT19zAdAd5Q4HSmLVA
orq_0000AT14A8ixxzU8mK5dQ0
orq_0000AT0yiVf1NwLJ7NaK8G
orq_0000AT0yiLoZzY2wa7yods


In [43]:
slices = [
  {
    "origin": "NYC",
    "destination": "ATL",
    "departure_date": "2023-03-21"
  },
  {
    "origin": "ATL",
    "destination": "NYC",
    "departure_date": "2023-03-21"
  }
]
passengers = [{ "type": "adult" }, { "type": "adult" }, { "age": 1 }]
client.offer_requests.create()\
  .slices(slices)\
  .passengers(passengers)\
  .cabin_class("business")\
  .execute()

OfferRequest(id='orq_0000AT23PKyBtmqYbqqjBY', live_mode=False, cabin_class='business', created_at=datetime.datetime(2023, 2, 25, 8, 21, 19, 634067), offers=[], slices=[OfferRequestSlice(destination_type='city', destination=City(id='cit_atl_us', name='Atlanta', iata_code='ATL', iata_country_code='US'), origin_type='city', origin=City(id='cit_nyc_us', name='New York', iata_code='NYC', iata_country_code='US'), departure_date=datetime.date(2023, 3, 21)), OfferRequestSlice(destination_type='city', destination=City(id='cit_nyc_us', name='New York', iata_code='NYC', iata_country_code='US'), origin_type='city', origin=City(id='cit_atl_us', name='Atlanta', iata_code='ATL', iata_country_code='US'), departure_date=datetime.date(2023, 3, 21))], passengers=[OfferRequestPassenger(id='pas_0000AT23PKyBtmqYbqqjBb', age=None, type='adult', given_name=None, family_name=None, loyalty_programme_accounts=[]), OfferRequestPassenger(id='pas_0000AT23PKyBtmqYbqqjBc', age=None, type='adult', given_name=None, fam

In [46]:
destination = 'ATL'
origin = 'NYC'
departure_date = '2023-03-21'
print("\nSearching flights...")
slices = [
    {
        "origin": origin,
        "destination": destination,
        "departure_date": departure_date,
    },
]
offer_request = (
    client.offer_requests.create()
    .passengers([{"type": "adult"}])
    .slices(slices)
    .return_offers()
    .execute()
)
offers = offer_request.offers
for idx, offer in enumerate(offers):
    print(
        f"{idx + 1}. {offer.owner.name} flight departing at "
        + f"{offer.slices[0].segments[0].departing_at} duration of "
         + f"{offer.slices[0].segments[0].duration} "
        + f"{offer.total_amount} {offer.total_currency}"
        )


Searching flights...
1. United Airlines flight departing at 2023-03-21 20:45:00 duration of PT2H31M 53.39 USD
2. United Airlines flight departing at 2023-03-21 16:59:00 duration of PT2H32M 53.39 USD
3. United Airlines flight departing at 2023-03-21 15:05:00 duration of PT2H31M 53.39 USD
4. United Airlines flight departing at 2023-03-21 13:29:00 duration of PT2H30M 53.39 USD
5. United Airlines flight departing at 2023-03-21 11:58:00 duration of PT2H30M 53.39 USD
6. United Airlines flight departing at 2023-03-21 10:40:00 duration of PT2H29M 53.39 USD
7. United Airlines flight departing at 2023-03-21 06:30:00 duration of PT2H31M 53.39 USD
8. American Airlines flight departing at 2023-03-21 14:09:00 duration of PT2H52M 53.90 USD
9. American Airlines flight departing at 2023-03-21 20:20:00 duration of PT2H51M 53.90 USD
10. American Airlines flight departing at 2023-03-21 08:30:00 duration of PT2H39M 53.90 USD
11. American Airlines flight departing at 2023-03-21 17:29:00 duration of PT2H39M

In [48]:
offer = offers[1]
print(
    f"{idx + 1}. {offer.owner.name} flight departing at "
    + f"{offer.slices[0].segments[0].departing_at} duration of "
      + f"{offer.slices[0].segments[0].duration} "
    + f"{offer.total_amount} {offer.total_currency}"
    )
break

350. United Airlines flight departing at 2023-03-21 16:59:00 duration of PT2H32M 53.39 USD


SyntaxError: ignored

In [49]:
import datetime

duration_string = "PT2H31M"
duration = datetime.timedelta(hours=int(duration_string[2]), minutes=int(duration_string[4:-1]))
total_minutes = duration.total_seconds() // 60
print(total_minutes)


151.0


In [50]:
integer_amount = float(offer.total_amount)
print(integer_amount)


53.39


In [51]:
offer = offers[1]
offer.owner.name

'United Airlines'

In [52]:
import datetime
#Getting the top airline details
offer = offers[1]

#Extracting the flight duration in minutes
duration_string = offer.slices[0].segments[0].duration
duration = datetime.timedelta(hours=int(duration_string[2]), minutes=int(duration_string[4:-1]))
total_minutes = duration.total_seconds() // 60
print(total_minutes)

#Extracting flight price
flight_price = float(offer.total_amount)
print(flight_price)


152.0
53.39


Kayak Datascraping


Webscraping using Beautiful Soup

In [ ]:
import time
import requests
from bs4 import BeautifulSoup
origin = "LAX"
destination = "SFO"
start_date = "2023-03-19"
base_url = "https://www.expedia.com/Flights-Search?flight-type=on&mode=search&trip=roundtrip&leg1=from%3ASan+Francisco+%28SFO+-+San+Francisco+Intl.%29%2Cto%3ALas+Vegas+%28LAS+-+Harry+Reid+Intl.%29%2Cdeparture%3A3%2F8%2F2023TANYT&options=cabinclass%3Aeconomy&leg2=from%3ALas+Vegas+%28LAS+-+Harry+Reid+Intl.%29%2Cto%3ASan+Francisco+%28SFO+-+San+Francisco+Intl.%29%2Cdeparture%3A3%2F9%2F2023TANYT&passengers=children%3A0%2Cadults%3A1%2Cseniors%3A0%2Cinfantinlap%3AY&fromDate=3%2F8%2F2023&toDate=3%2F9%2F2023&d1=2023-03-08&d2=2023-03-09"
url = f"{base_url}/{origin}-{destination}/{start_date}?sort=bestflight_a"
url
response = requests.get(base_url)
time.sleep(1)
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup

In [ ]:
from selenium.common.exceptions import NoSuchElementException

# Close cookies pop-up
accept_cookies_xpath = '/html/body/div[12]/div/div[3]/div/div/div/div/div[1]/div/div[2]/div[2]/div[1]/button/span'

try:
   requests.find_element_by_xpath(accept_cookies_xpath).click()
except NoSuchElementException:
   pass

Webscraping using selenium

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver
from selenium import webdriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--diable-dve-shm-uage')

driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options = options)

In [ ]:
import requests

url = "https://skyscanner50.p.rapidapi.com/api/v1/searchAirport"

querystring = {"query":"london"}

headers = {
	"X-RapidAPI-Key": "e58dd894f7mshdb83450cb50f8c9p172875jsn5bbc9b76f61e",
	"X-RapidAPI-Host": "skyscanner50.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message":"You are not subscribed to this API."}
